In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import sklearn as skl
import tensorflow as tf
import seaborn as sns
import sklearn as skl
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

In [2]:
stock = 'EOS-USD'
api_key = "QO234RMNT7YYSXZ3"
period = 60
ts = TimeSeries(key=api_key, output_format="pandas")
data_ts = ts.get_intraday(stock.upper(), interval="60min", outputsize="compact")
data_ts

(                     1. open  2. high  3. low  4. close   5. volume
 date                                                               
 2020-06-17 19:39:03   2.5626   2.5626  2.5626    2.5626         0.0
 2020-06-17 19:00:00   2.5600   2.5700  2.5600    2.5600  11598950.0
 2020-06-17 17:00:00   2.5500   2.5600  2.5500    2.5600         0.0
 2020-06-17 16:00:00   2.5400   2.5600  2.5300    2.5500  28795008.0
 2020-06-17 15:00:00   2.5500   2.5500  2.5400    2.5400         0.0
 ...                      ...      ...     ...       ...         ...
 2020-06-13 20:00:00   2.6000   2.6100  2.6000    2.6100         0.0
 2020-06-13 19:00:00   2.6000   2.6100  2.6000    2.6000         0.0
 2020-06-13 18:00:00   2.5946   2.6014  2.5946    2.6011         0.0
 2020-06-13 17:00:00   2.5900   2.6000  2.5900    2.5900   2168275.0
 2020-06-13 16:00:00   2.5900   2.6000  2.5900    2.5900  18056774.0
 
 [100 rows x 5 columns],
 {'1. Information': 'Intraday (60min) open, high, low, close prices and volu

In [3]:
data_ts[0]

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-06-17 19:39:03,2.5626,2.5626,2.5626,2.5626,0.0
2020-06-17 19:00:00,2.5600,2.5700,2.5600,2.5600,11598950.0
2020-06-17 17:00:00,2.5500,2.5600,2.5500,2.5600,0.0
2020-06-17 16:00:00,2.5400,2.5600,2.5300,2.5500,28795008.0
2020-06-17 15:00:00,2.5500,2.5500,2.5400,2.5400,0.0
...,...,...,...,...,...
2020-06-13 20:00:00,2.6000,2.6100,2.6000,2.6100,0.0
2020-06-13 19:00:00,2.6000,2.6100,2.6000,2.6000,0.0
2020-06-13 18:00:00,2.5946,2.6014,2.5946,2.6011,0.0


In [4]:
open_p = list(data_ts[0]['1. open'].to_numpy())
close_p = list(data_ts[0]['4. close'].to_numpy())
high_p = list(data_ts[0]['2. high'].to_numpy())
low_p = list(data_ts[0]['3. low'].to_numpy())
result = []
counts = []
def getPivots(data):
    all_data = []
    for x in data:
        all_data.append( [ x, data.count(x)] )
        counts.append(data.count(x))
    unrepeated = []
    for x in all_data:
        if x not in unrepeated:
            unrepeated.append(x)
    sorted_result = sorted(unrepeated, key = lambda i: i[1], reverse=True)
    arr = np.array(sorted_result)

    for x in list(arr):
        if x[1] > np.quantile(np.array(counts), 0.8):
            result.append(x)
    return result

r_s_open = getPivots(open_p)
r_s_close = getPivots(close_p)
r_s_high = getPivots(high_p)
r_s_low = getPivots(low_p)
r_s_open

[array([ 2.55, 18.  ]),
 array([ 2.55, 17.  ]),
 array([ 2.55, 16.  ]),
 array([ 2.55, 16.  ])]

In [5]:
r_s_close

[array([ 2.55, 18.  ]),
 array([ 2.55, 17.  ]),
 array([ 2.55, 16.  ]),
 array([ 2.55, 16.  ])]

In [6]:
r_s_low

[array([ 2.55, 18.  ]),
 array([ 2.55, 17.  ]),
 array([ 2.55, 16.  ]),
 array([ 2.55, 16.  ])]

In [7]:
r_s_high

[array([ 2.55, 18.  ]),
 array([ 2.55, 17.  ]),
 array([ 2.55, 16.  ]),
 array([ 2.55, 16.  ])]

In [8]:
data_ts[0].take([4], axis=1)

,5. volume
date,
2020-06-17 19:39:03,0.0
2020-06-17 19:00:00,11598950.0
2020-06-17 17:00:00,0.0
2020-06-17 16:00:00,28795008.0
2020-06-17 15:00:00,0.0
...,...
2020-06-13 20:00:00,0.0
2020-06-13 19:00:00,0.0
2020-06-13 18:00:00,0.0


In [15]:
def r_s_levels(data):
    for x in range(4):
        info = data.take([x], axis=1).to_numpy()
        data = getPivots(list(info))
        print( data )
r_s_levels(data_ts[0])

[array([ 2.55, 18.  ]), array([ 2.55, 17.  ]), array([ 2.55, 16.  ]), array([ 2.55, 16.  ]), array([array([2.55]), 18], dtype=object)]


AttributeError: 'list' object has no attribute 'take'